# Load libraries and data:
The data will be the one exported from the 'exploration_and_feature_engineering' file

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
train = pd.read_csv("train_modified.csv")
test = pd.read_csv("test_modified.csv")

In [3]:
print train.shape
train.dtypes

(8523, 34)


Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Visibility              float64
Item_Weight                  float64
Outlet_Identifier             object
Item_Visibility_MeanRatio    float64
Outlet_Years                   int64
Item_Fat_Content_0           float64
Item_Fat_Content_1           float64
Item_Fat_Content_2           float64
Outlet_Location_Type_0       float64
Outlet_Location_Type_1       float64
Outlet_Location_Type_2       float64
Outlet_Size_0                float64
Outlet_Size_1                float64
Outlet_Size_2                float64
Outlet_Type_0                float64
Outlet_Type_1                float64
Outlet_Type_2                float64
Outlet_Type_3                float64
Item_Type_Combined_0         float64
Item_Type_Combined_1         float64
Item_Type_Combined_2         float64
Outlet_0                     float64
Outlet_1                     float64
Outlet_2                     float64
O

## Baseline models:

In [5]:
#Mean based:
mean_sales = train['Item_Outlet_Sales'].mean()

#Define a dataframe with IDs for submission:
base1 = test[['Item_Identifier','Outlet_Identifier']]
base1['Item_Outlet_Sales'] = mean_sales

#Export submission file
base1.to_csv("alg0.csv",index=False)

/Users/aarshay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Function to fit and generate submission file:

In [9]:
#Define target and ID columns:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn import cross_validation, metrics
def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print "\nModel Report"
    print "RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions))
    print "CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

## Linear Regression Model:

In [11]:
from sklearn.linear_model import LinearRegression, Ridge

In [12]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = LinearRegression(normalize=True)
modelfit(alg1, train, test, predictors, target, IDcol, 'alg1.csv')
print '\nCoefficients:'
print pd.Series(alg1.coef_, predictors)


Model Report
RMSE : 1128
CV Score : Mean - 1129 | Std - 44.17 | Min - 1074 | Max - 1218

Coefficients:
Item_MRP                     1.557723e+01
Item_Visibility             -2.747623e+02
Item_Weight                 -4.096153e-01
Item_Visibility_MeanRatio    1.000084e+02
Outlet_Years                 1.002508e+15
Item_Fat_Content_0           1.982563e+16
Item_Fat_Content_1           8.532088e+15
Item_Fat_Content_2           1.982563e+16
Outlet_Location_Type_0      -1.286406e+16
Outlet_Location_Type_1       2.429911e+16
Outlet_Location_Type_2      -1.820205e+16
Outlet_Size_0                2.422277e+16
Outlet_Size_1                9.329826e+15
Outlet_Size_2               -8.972439e+15
Outlet_Type_0                1.107313e+16
Outlet_Type_1                6.188985e+15
Outlet_Type_2                7.895853e+15
Outlet_Type_3                5.590700e+15
Item_Type_Combined_0        -5.388532e+15
Item_Type_Combined_1        -5.388532e+15
Item_Type_Combined_2         5.905014e+15
Outlet_0      

## Ridge Regression Model:

In [37]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg2 = Ridge(alpha=0.05,normalize=True)
modelfit(alg2, train, test, predictors, target, IDcol, 'alg2.csv')
print '\nCoefficients:'
print pd.Series(alg2.coef_, predictors)


Model Report
RMSE : 1129
CV Score : Mean - 1130 | Std - 44.6 | Min - 1076 | Max - 1217

Coefficients:
Item_MRP                      14.809236
Item_Visibility             -272.632195
Item_Weight                    0.159405
Item_Visibility_MeanRatio    -85.380187
Outlet_Years                   3.805083
Item_Fat_Content_0           -14.766820
Item_Fat_Content_1           -14.555244
Item_Fat_Content_2            25.782327
Outlet_Location_Type_0      -113.544031
Outlet_Location_Type_1       109.775614
Outlet_Location_Type_2        -5.238874
Outlet_Size_0                -29.654098
Outlet_Size_1                200.037150
Outlet_Size_2               -167.383803
Outlet_Type_0               -775.566903
Outlet_Type_1                168.360688
Outlet_Type_2               -154.572323
Outlet_Type_3                644.481828
Item_Type_Combined_0          -9.352783
Item_Type_Combined_1          14.901432
Item_Type_Combined_2         -14.555244
Outlet_0                    -760.699665
Outlet_1         

## Decision Tree Model:

In [16]:
from sklearn.tree import DecisionTreeRegressor

In [17]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg3 = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)
modelfit(alg3, train, test, predictors, target, IDcol, 'alg3.csv')
print '\nFeature Importances:'
print pd.Series(alg3.feature_importances_, predictors)


Model Report
RMSE : 1058
CV Score : Mean - 1091 | Std - 45.42 | Min - 1003 | Max - 1186

Feature Importances:
Item_MRP                     0.541850
Item_Visibility              0.003911
Item_Weight                  0.002238
Item_Visibility_MeanRatio    0.004326
Outlet_Years                 0.026579
Item_Fat_Content_0           0.001099
Item_Fat_Content_1           0.000000
Item_Fat_Content_2           0.000000
Outlet_Location_Type_0       0.000000
Outlet_Location_Type_1       0.000000
Outlet_Location_Type_2       0.000528
Outlet_Size_0                0.000000
Outlet_Size_1                0.000118
Outlet_Size_2                0.000091
Outlet_Type_0                0.318395
Outlet_Type_1                0.004140
Outlet_Type_2                0.000000
Outlet_Type_3                0.000000
Item_Type_Combined_0         0.000000
Item_Type_Combined_1         0.000000
Item_Type_Combined_2         0.000000
Outlet_0                     0.000000
Outlet_1                     0.000000
Outlet_2       

In [21]:
predictors = ['Item_MRP','Outlet_Type_0','Outlet_5','Outlet_Years']
alg4 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=150)
modelfit(alg4, train, test, predictors, target, IDcol, 'alg4.csv')
print '\nFeature Importances:'
print pd.Series(alg4.feature_importances_, predictors)


Model Report
RMSE : 1071
CV Score : Mean - 1096 | Std - 43.3 | Min - 1027 | Max - 1172

Feature Importances:
Item_MRP         0.548508
Outlet_Type_0    0.323598
Outlet_5         0.003372
Outlet_Years     0.124522
dtype: float64


## Random Forest Model:
Note: random forest models are not 100% replicable. So the outputs might differ very slightly but should be around the ballpark.

In [23]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg5 = RandomForestRegressor(n_estimators=200,max_depth=5, min_samples_leaf=100,n_jobs=4)
modelfit(alg5, train, test, predictors, target, IDcol, 'alg5.csv')
print '\nFeature Importances:'
print pd.Series(alg5.feature_importances_, predictors).sort_values()


Model Report
RMSE : 1073
CV Score : Mean - 1083 | Std - 43.9 | Min - 1020 | Max - 1161

Feature Importances:
Outlet_9                     0.000000e+00
Item_Type_Combined_2         0.000000e+00
Item_Type_Combined_0         0.000000e+00
Outlet_8                     0.000000e+00
Outlet_6                     0.000000e+00
Outlet_Size_2                0.000000e+00
Outlet_1                     0.000000e+00
Outlet_Size_0                0.000000e+00
Item_Fat_Content_1           0.000000e+00
Outlet_7                     0.000000e+00
Outlet_0                     4.474509e-07
Outlet_Location_Type_0       6.947994e-07
Outlet_4                     7.637379e-07
Item_Fat_Content_0           1.241221e-06
Item_Weight                  2.343842e-06
Item_Fat_Content_2           3.144387e-06
Item_Type_Combined_1         9.012653e-06
Outlet_Location_Type_1       1.489440e-05
Outlet_Location_Type_2       1.882341e-05
Outlet_2                     2.108708e-05
Outlet_Size_1                3.807067e-05
Item_Vis

In [57]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg6 = RandomForestRegressor(n_estimators=400,max_depth=6, min_samples_leaf=100,n_jobs=4)
modelfit(alg6, train, test, predictors, target, IDcol, 'alg6.csv')
print '\nFeature Importances:'
print pd.Series(alg6.feature_importances_, predictors)


Model Report
RMSE : 1068
CV Score : Mean - 1083 | Std - 43.63 | Min - 1019 | Max - 1161

Feature Importances:
Item_MRP                     5.495375e-01
Item_Visibility              1.111009e-03
Item_Weight                  8.781811e-04
Item_Visibility_MeanRatio    9.782462e-04
Outlet_Years                 5.353328e-02
Item_Fat_Content_0           1.939206e-04
Item_Fat_Content_1           1.028131e-05
Item_Fat_Content_2           3.643779e-05
Outlet_Location_Type_0       5.787375e-05
Outlet_Location_Type_1       3.407967e-04
Outlet_Location_Type_2       6.347284e-04
Outlet_Size_0                0.000000e+00
Outlet_Size_1                2.181069e-04
Outlet_Size_2                7.833316e-05
Outlet_Type_0                3.221834e-01
Outlet_Type_1                4.724733e-04
Outlet_Type_2                3.190431e-04
Outlet_Type_3                3.281459e-02
Item_Type_Combined_0         0.000000e+00
Item_Type_Combined_1         1.891201e-05
Item_Type_Combined_2         3.813132e-05
Outlet_